## Importing libraries

In [1]:
import sys
import json
import nanohubuidl.teleport as t
from nanohubuidl.plotly import PlotlyBuilder
from nanohubuidl.rappture import RapptureBuilder
from nanohubuidl.material import MaterialBuilder
from nanohubuidl.material import MaterialContent
from nanohubuidl.app import AppBuilder

from nanohubuidl.nanohub import Auth
TOOLNAME="bandstrlab"
STATE_LOADER_STATUS = "loader_status"
STATE_LOADER_OPEN = "loader_open"
STATE_ERROR_STATUS = "error_status"
STATE_ERROR_OPEN = "error_open"


## Creating the project and adding GUI variables 

In [2]:
Project = t.TeleportProject("Bandstructures Lab")
Component = Project.root
Component.addStateVariable("DrawerIsVisible", {"type":"boolean", "defaultValue": True})
Component.addStateVariable("open_results", {"type":"boolean", "defaultValue": False})
Component.addStateVariable("open_params", {"type":"boolean", "defaultValue": True})

## Getting session information

In [3]:
import os
auth_data = {
    'grant_type' : 'tool',
}
with open(os.environ["SESSIONDIR"]+"/resources") as file:
    lines = [line.split(" ", 1) for line in file.readlines()]
    properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
    auth_data["sessiontoken"] = properties["session_token"]
    auth_data["sessionnum"] = properties["sessionid"]

## Creating Error

In [4]:
ErrorMessage = RapptureBuilder.Error( 
    Component, 
    error_status = STATE_ERROR_STATUS,
    error_open = STATE_ERROR_OPEN,
)

## Creating Loader

In [5]:
Loader = RapptureBuilder.Loader( 
    Component, 
    loader_status = STATE_LOADER_STATUS,
    loader_open = STATE_LOADER_OPEN,
)

## Create Tool Schema

In [6]:
RapptureBuilder.buildSchema(
    Project,
    Component,
    url = "https://nanohub.org/api/tools",
    toolname = TOOLNAME
)
Component.addPropVariable("onLoadSchema", {"type":"func", 'defaultValue' : '(e)=>{e.setState({"'+ STATE_LOADER_OPEN +'":false})}'})    

## Create Session Authentication

In [7]:
Login, CLogin = Auth.Session(
    Project,
    Component,
    sessiontoken = auth_data["sessiontoken"],
    sessionnum = auth_data["sessionnum"],
    url = "https://nanohub.org/api/developer/oauth/token",   
)
Login.content.events["onError"]=[
    { "type": "stateChange", "modifies": STATE_ERROR_OPEN, "newState": True},
    { "type": "stateChange", "modifies": STATE_ERROR_STATUS, "newState": '$e'},
]
Login.content.events["onAuth"] = [
    { "type": "propCall2", "calls": "buildSchema", "args": ['self'] }            
]


In [8]:
Drawer = MaterialBuilder.Drawer(Project, state="DrawerIsVisible", variant="persistent" )

## Defining Input Parameters

In [9]:
SETTINGS = {
    'Type_of_calculation': { 
        'type' : 'Select', 
        'description' : 'Choose what kind of bandstructure calculation is required electron/phonon', 
        "units" : "",         
        'label' : 'Calculation For', 
        'default_value' : 'Electron', 
        'options' : {'Phonon':'Phonons','Electron':'Electrons'}, 
    },
    'material': {
        'type' : 'Select', 
        'description' : 'Choose the material the device is made of', 
        'label' : 'Material', 
        'default_value' : 'Si', 
        'options' : {'Si':'Si', 'Ge':'Ge', 'GaAs':'GaAs', 'InAs':'InAs', 'AlGaAs':'AlGaAs', 'SiGe':'SiGe'},
        "units" : "",         
    },
    'alloy_model': {
        'type' : 'Select', 
        'description' : 'Choose how alloy materials should be modeled', 
        'label' : 'Alloy Model', 
        'default_value' : 'VCA', 
        'options' : {'VCA':'Virtual Crystal Approximation','RAND_ALLOY':'Random Alloy'},
        "units" : "",         
    },
    'molar_fraction': {
        'type' : 'Number',
        'description' : 'This is the molar fraction of Al in AlGaAs or Si in SiGe', 
        'label' : 'Molar fraction', 
        'default_value' : 0.5, 
        'min' : 0.0, 
        'max' : 1.0, 
        "units" : "",         
    },
    'geom': {
        'type' : 'Select', 
        'label' : '', 
        'description' : '', 
        'default_value' : 'bulk', 
        'options' : {'bulk':'Bulk','utb':'UTB','wire':'Wire'},

    },
    'pmodel1': {
        'type' : 'Select', 
        'description' : 'Choose phonon dispersion model', 
        'label' : 'Model', 
        'default_value' : 'Keating', 
        'options' : {'Keating':'Keating','Modified Valence Force Field':'Modified Valence Force Field'},
        "units" : "",         
    },
    'pmodel2': {
        'type' : 'Select', 
        'label' : 'Choose phonon dispersion model', 
        'description' : 'Choose phonon dispersion model', 
        'default_value' : 'Keating', 
        'options' : {'Keating':'Keating'},
        "units" : "",         
    },
    'tbinding': {
        'type' : 'Select', 
        'description' : 'Choose tight binding basis', 
        'label' : 'Tight Binding Model:', 
        'default_value' : 'sp3d5sstar', 
        'options' : {'sp3d5sstar':'sp3d5s*'}, 
    },
    'spin': {
        'type' : 'Boolean', 
        'description' : 'Enable spin orbit coupling in bandstructure calculation', 
        'label' : 'Spin-Orbit Coupling:', 
        'default_value' : True, 
    },
    'model': {
        'type' : 'Select', 
        'description' : 'Choose type of strain applied', 
        'label' : 'Strain type:', 
        'default_value' : 'none', 
        'options' : {'none':'None','uniaxial':'Uniaxial', 'biaxial':'Biaxial', 'freeform':'Free Form','hydrostatic':'Hydrostatic'}, 
    },
    'sweep': {
        'type' : 'Boolean', 
        'description' : 'Perform a sweep over strain values over a range', 
        'label' : 'Perform sweep over entire range?', 
        'default_value' : False, 
    },
    'epsilon': {
        'type' : 'Number', 
        'description' : 'Please enter strain ratio', 
        'label' : 'Epsilon',
        'default_value' : '0.01', 
        'min' : -0.05, 
        'max' : 0.05, 
        'units': '',
    },
    'eps1': {
        'type' : 'Number', 
        'description' : 'Please enter starting value of strain ratio sweep', 
        'label' : 'Starting value of epsilon:', 
        'default_value' : -0.05, 
        'min' : -0.05, 
        'max' : 0.05, 
        'units': '',
    },
    'eps2': {
        'type' : 'Number',
        'description' : 'Please enter ending value of strain ratio sweep', 
        'label' : 'Ending value of epsilon:', 
        'default_value' : 0.05, 
        'min' : -0.05, 
        'max' : 0.05, 
        'units': '',
    },
    'num_eps': {
        'type' : 'Number', 
        'description' : '', 
        'label' : 'Number of sweep points:', 
        'default_value' : 11.0, 
        'min' : 2.0, 
        'max' : 101.0, 
        'units': '',
    },
    'x': {
        'type' : 'Integer', 
        'description' : 'Miller Index of strain  direction along x', 
        'label' : 'Miller index along x:', 
        'default_value' : 1, 
        'min' : -5, 
        'max' : 5, 
        'units': '',
    },
    'y': {
        'type' : 'Integer', 
        'description' : 'Miller Index of strain  direction along y', 
        'label' : 'Miller index along y:', 
        'default_value' : 0, 
        'min' : -5, 
        'max' : 5, 
        'units': '',
    },
    'z': {
        'type' : 'Integer', 
        'description' : 'Miller Index of strain  direction along z', 
        'label' : 'Miller index along z:', 
        'default_value' : 0, 
        'min' : -5, 
        'max' : 5,
        'units': '',
    },
    'exx': {
        'type' : 'Number', 
        'description' : 'Strain  ratio along x', 
        'label' : 'exx', 
        'default_value' : 0.01, 
        'min' : -0.05, 
        'max' : 0.05, 
        'units': '',
    },
    'eyy': {
        'type' : 'Number', 
        'description' : 'Strain  ratio along y', 
        'label' : 'eyy', 
        'default_value' : 0.01, 
        'min' : -0.05, 
        'max' : 0.05, 
        'units': '',
    },
    'ezz': {
        'type' : 'Number', 
        'description' : 'Strain  ratio along z', 
        'label' : 'ezz', 
        'default_value' : 0.01, 
        'min' : -0.05, 
        'max' : 0.05, 
        'units': '',
    },
    'exy': {
        'type' : 'Number', 
        'description' : 'Strain  ratio along xy', 
        'label' : 'exy', 
        'default_value' : 0.01, 
        'min' : -0.05, 
        'max' : 0.05, 
        'units': '',
    },
    'exz': {
        'type' : 'Number', 
        'description' : 'Strain  ratio along xz', 
        'label' : 'exz', 
        'default_value' : 0.01, 
        'min' : -0.05, 
        'max' : 0.05, 
        'units': '',
    },
    'eyz': {
        'type' : 'Number', 
        'description' : 'Strain  ratio along yz', 
        'label' : 'eyz', 
        'default_value' : 0.01, 
        'min' : -0.05, 
        'max' : 0.05, 
        'units': '',
    },
    'dangling': {
        'type' : 'Number', 
        'description' : '', 
        'label' : 'Dangling Bond Energy:', 
        'units' : 'eV', 
        'default_value' : 30, 
        'min' : 0.0, 
        'max' : 1000.0, 
        'units': '',
    },
    'which_bands': {
        'type' : 'Select', 
        'description' : 'Compute these energy bands.', 
        'label' : 'Explore:', 
        'default_value' : 'CB', 
        'options' : {'CB':'Conduction bands','VB':'Valence bands','BB':'Both bands','manual':'Manual energy target'}, 
    },
    'shift': {
        'type' : 'Number', 
        'description' : 'Target energy for eigenstates.', 
        'label' : 'Target:', 
        'units' : 'eV', 
        'default_value' : 0.5, 
        'min' : -10.0, 
        'max' : 10.0, 
        'units': '',
    },
    'eps': {
        'type' : 'Number', 
        'description' : 'This is the amount of (unitless) strain applied, typically a few percent.  If negative it is compressive, and expansive otherwise.', 
        'label' : 'Epsilon', 
        'default_value' : 0.02, 
        'min' : -0.05, 
        'max' : 0.05, 
        'units': '',
    },
    'dev_type': {
        'type' : 'Select', 
        'description' : '', 
        'label' : 'Select DGMOS type:', 
        'default_value' : 'NMOS', 
        'options' : {'NMOS':'N-type DGMOS','PMOS':'P-type DGMOS'}, 
    },
    'Tel': {
        'type' : 'Number', 
        'description' : 'The temperature at which the simulation is done. This controls the spreading of Fermi-Dirac function which in turn controls the filling of bands in the nanodevice.', 
        'label' : 'Electron temperature :', 
        'units' : 'K', 
        'default_value' : 300, 
        'min' : 4.0, 
        'max' : 1000.0, 
    },
    'gate_sweep': {
        'type' : 'Boolean',
        'id' : 'gate_sweep', 
        'description' : '', 
        'label' : 'Perform sweep over range of Gate Voltage ?', 
        'default_value' : False, 
    },
    'VG': {
        'type' : 'Number', 
        'description' : '', 
        'label' : 'Front gate bias(V) :', 
        'units' : 'V', 
        'default_value' : 1.25, 
        'min' : -2.0, 
        'max' : 4.0, 
    },
    'vgmin': {
        'type' : 'Number', 
        'description' : '', 
        'label' : 'Initial front gate bias(V) :', 
        'units' : 'V', 
        'default_value' : 0.2, 
        'min' : -2.0, 
        'max' : 2.0, 
    },
    'vgmax': {
        'type' : 'Number', 
        'description' : '', 
        'label' : 'Final front gate bias(V) :', 
        'units' : 'V', 
        'default_value' : 0.2, 
        'min' : -2.0, 
        'max' : 2.0, 
    },
    'vgdiv': {
        'type' : 'Number', 
        'description' : '', 
        'label' : 'Number of gate bias points:', 
        'default_value' : 3, 
        'min' : 3.0, 
        'max' : 30.0, 
        'units' : '', 
    },
    'VD': {
        'type' : 'Number', 
        'description' : '', 
        'label' : 'Drain bias(V) :', 
        'units' : 'V', 
        'default_value' : 0.05, 
        'min' : -1.0, 
        'max' : 1.0, 
    },
    'workfunction': {
        'type' : 'Number', 
        'id' : 'workfunction', 
        'description' : 'This is the workfunction of the gate material, like the metal or polysilicon used for making the device gate.', 
        'label' : 'Gate workfunction :', 
        'units' : 'eV', 
        'default_value' : 4.25, 
        'min' : 0.0, 
        'max' : 5.0, 
    },
    'el_aff': {
        'type' : 'Number', 
        'description' : 'This is the electron affinity of the channel semiconductor material', 
        'label' : 'Electron affinity :', 
        'units' : 'eV', 
        'default_value' : 4.05, 
        'min' : 0.0, 
        'max' : 10.0, 
    },
    'config': {
        'type' : 'Select', 
        'id' : 'config', 
        'description' : 'Chose how back gate is connected', 
        'label' : '', 
        'default_value' : 'TFG', 
        'options' : {'TFG':'Connected to front gate', 'FGB':'Fixed bias'}, 
    },
    'Vback': {
        'type' : 'Number', 
        'description' : '', 
        'label' : 'Back gate bias :', 
        'units' : 'V', 
        'default_value' : 0.01, 
        'min' : 0.0, 
        'max' : 2.0, 
    },
    'EF_method': {
        'type' : 'Select', 
        'id' : 'EF_method', 
        'description' : 'Select the way to specify the Fermilevel', 
        'label' : 'Fermilevel method:', 
        'default_value' : 'EF', 
        'options' : {'EF':'Specify Source Fermi level'}, 
    },
    'EF': {
        'type' : 'Number', 
        'id' : 'EF', 
        'description' : 'Set the fermi-level for the source, this indicates the doping in the source. This value is set with respect to the bulk conduction band minima for NMOS and bulk valence band maxima for PMOS.', 
        'label' : 'Fermilevel:', 
        'units' : 'V', 
        'default_value' : 0.44, 
        'min' : -2.0, 
        'max' : 2.0, 
    },
    'choice_Symmetry_point': {
        'type' : 'Select', 
        'id' : 'choice_Symmetry_point', 
        'description' : 'Specify the second point from Gamma for which bandstructure is to be plotted', 
        'label' : 'High Symmetry Point From Gamma', 
        'default_value' : '[1 0 0]', 
        'options' : {'[1 0 0]':'[1 0 0]','[1 1 0]':'[1 1 0]','[1 1 1]':'[1 1 1]'}, 
    },
    'path': {
        'type' : 'Select', 
        'description' : '', 
        'label' : 'Choose a path', 
        'default_value' : 'path', 
        'options' : {'path':'Path Between High Symmetry Points','freeform':'Free Form Path','volume':'Volume'}, 
    },
    'choice1': {
        'type' : 'Select', 
        'description' : '', 
        'label' : 'High Symmetry Point', 
        'default_value' : 'Gamma', 
        'options' : {'Gamma':'Gamma','L    ':'L    ','X    ':'X    ','K    ':'K    ','W    ':'W    ','U    ':'U    '}, 
    },
    'L': {
        'type' : 'Select',
        'description' : '', 
        'label' : 'L point', 
        'default_value' : '(0.5, 0.5, 0.5)', 'options' : {
            '(0.5, 0.5, 0.5)':'{ 0.5, 0.5, 0.5} ',
            '(0, 0.5, 0)':'{ 0.5,  0.5, -0.5}', 
            '(0.5, 0, 0)':'{ 0.5, -0.5,  0.5}', 
            '(0, 0, -0.5)':'{ 0.5, -0.5, -0.5}', 
            '(0, 0, 0.5)':'{-0.5,  0.5,  0.5}', 
            '(-0.5, 0, 0)':'{-0.5,  0.5, -0.5}', 
            '(0, -0.5, 0)':'{-0.5, -0.5,  0.5}', 
            '(-0.5, -0.5, -0.5)':'{-0.5, -0.5, -0.5}'
        },
    },
    'X': {
        'type' : 'Select',
        'description' : '', 
        'label' : 'X point', 
        'default_value' : '(0.5, 0.5, 0)', 
        'options' : {
            '(0.5, 0.5, 0)':'{1, 0, 0}',
            '(0, 0.5, 0.5)':'{0, 1, 0}',
            '(0.5, 0, 0.5)':'{0, 0, 1}',
            '(-0.5, 0, -0.5)':'{0, 0, -1}',
            '(0, -0.5, -0.5)':'{0, -1, 0}',
            '(-0.5, -0.5, 0)':'{-1, 0, 0}',
        }
    },
    'K': {
        'type' : 'Select',
        'description' : '', 
        'label' : 'K point', 
        'default_value' : '(0.375, 0.75, 0.375)', 
        'options' : {
            '(0.375, 0.75, 0.375)':'{ 0.75, 0.75, 0   }',
            '(0.75, 0.375, 0.375)':'{ 0.75, 0   , 0.75}',
            '(0, 0.375, -0.375)':'{ 0.75, 0   ,-0.75}',
            '(0.375, 0, -0.375)':'{ 0.75,-0.75, 0   }',
            '(0.375, 0.375, 0.75)':'{ 0   , 0.75, 0.75}',
            '(-0.375, 0.375, 0)':'{ 0   , 0.75,-0.75}',
            '(0.375, -0.375, 0)':'{ 0   ,-0.75, 0.75}',
            '(-0.375, -0.375, -0.75)':'{ 0   ,-0.75,-0.75}',
            '(-0.375, 0, 0.375)':'{-0.75, 0.75, 0   }',
            '(0, -0.375, 0.375)':'{-0.75, 0   , 0.75}',
            '(-0.75, -0.375, -0.375)':'{-0.75, 0   ,-0.75}',
            '(-0.375, -0.75, -0.375)':'{-0.75,-0.75, 0   }',
        },
    },
    'W': {
        'type' : 'Select',
        'description' : '', 
        'label' : 'W point', 
        'default_value' : '(0.5, 0.75, 0.25)', 
        'options' : {
            '(0.5, 0.75, 0.25)':'{ 1  , 0.5, 0  }',
            '(0.75, 0.5, 0.25)':'{ 1  , 0  , 0.5}',
            '(0.25, 0.5, -0.25)':'{ 1  , 0  ,-0.5}',
            '(0.5, 0.25, -0.25)':'{ 1  ,-0.5, 0  }',
            '(0.25, 0.75, 0.5)':'{ 0.5, 1  , 0  }',
            '(0.75, 0.25, 0.5)':'{ 0.5, 0  , 1  }',
            '(0.25, 0.5, 0.75)':'{ 0  , 1  , 0.5}',
            '(0.5, 0.25, 0.75)':'{ 0  , 0.5, 1  }',
            '(-0.25, 0.25, -0.5)':'{ 0.5, 0  ,-1  }',
            '(0.25, -0.25, -0.5)':'{ 0.5,-1  , 0  }',
            '(-0.25, 0.5, 0.25)':'{ 0  , 1  ,-0.5}',
            '(0.5, -0.25, 0.25)':'{ 0  ,-0.5, 1  }',
            '(-0.5, 0.25, -0.25)':'{ 0  , 0.5,-1  }',
            '(0.25, -0.5, -0.25)':'{ 0  ,-1  , 0.5}',
            '(-0.25, 0.25, 0.5)':'{-0.5, 1  , 0  }',
            '(0.25, -0.25, 0.5)':'{-0.5, 0  , 1  }',
            '(-0.5, -0.25, -0.75)':'{ 0  ,-0.5,-1  }',
            '(-0.25, -0.5, -0.75)':'{ 0  ,-1  ,-0.5}',
            '(-0.75, -0.25, -0.5)':'{-0.5, 0  ,-1  }',
            '(-0.25, -0.75, -0.5)':'{-0.5,-1  , 0  }',
            '(-0.5, -0.25, 0.25)':'{-1  , 0.5, 0  }',
            '(-0.25, -0.5, 0.25)':'{-1  , 0  , 0.5}',
            '(-0.75, -0.5, -0.25)':'{-1  , 0  ,-0.5}',
            '(-0.5, -0.75, -0.25)':'{-1  ,-0.5, 0  }',
        },
    },
    'U': {
        'type' : 'Select',
        'description' : '', 
        'label' : 'U point', 
        'default_value' : '(0.625, 0.625, 0.25)', 
        'options' : {
            '(0.625, 0.625, 0.25)':'{ 1   , 0.25, 0.25}',
            '(0.375, 0.625, 0)':'{ 1   , 0.25,-0.25}',
            '(0.625, 0.375, 0)':'{ 1   ,-0.25, 0.25}',
            '(0.375, 0.375, -0.25)':'{ 1   ,-0.25,-0.25}',
            '(0.25, 0.625, 0.625)':'{ 0.25, 1   , 0.25}',
            '(0.625, 0.25, 0.625)':'{ 0.25, 0.25, 1   }',
            '(0, 0.625, 0.375)':'{ 0.25, 1   ,-0.25}',
            '(0.625, 0, 0.375)':'{ 0.25,-0.25, 1   }',
            '(0.375, 0, 0.625)':'{-0.25, 0.25, 1   }',
            '(0, 0.375, 0.625)':'{-0.25, 1   , 0.25}',
            '(-0.375, 0.25, -0.375)':'{ 0.25, 0.25,-1   }',
            '(0.25, -0.375, -0.375)':'{ 0.25,-1   , 0.25}',
            '(-0.25, 0.375, 0.375)':'{-0.25, 1   ,-0.25}',
            '(0.375, -0.25, 0.375)':'{-0.25,-0.25, 1   }',
            '(0, -0.375, -0.625)':'{ 0.25,-1   ,-0.25}',
            '(-0.375, 0, -0.625)':'{ 0.25,-0.25,-1   }',
            '(-0.625, 0, -0.375)':'{-0.25, 0.25,-1   }',
            '(0, -0.625, -0.375)':'{-0.25,-1   , 0.25}',
            '(-0.625, -0.25, -0.625)':'{-0.25,-0.25,-1   }',
            '(-0.25, -0.625, -0.625)':'{-0.25,-1   ,-0.25}',
            '(-0.375, -0.375, 0.25)':'{-1   , 0.25, 0.25}',
            '(-0.625, -0.375, 0)':'{-1   , 0.25,-0.25}',
            '(-0.375, -0.625, 0)':'{-1   ,-0.25, 0.25}',
            '(-0.625, -0.625, -0.25)':'{-1   ,-0.25,-0.25}',
        },
    },
    'choice2': {
        'type' : 'Select', 
        'description' : '', 
        'label' : 'Point 2', 
        'default_value' : 'Gamma', 
        'options' : {'Gamma':'Gamma','L    ':'L    ','X    ':'X    ','K    ':'K    ','W    ':'W    ','U    ':'U    '}, 
    },
    'check3': {
        'type' : 'Boolean', 
        'id' : 'check3', 
        'description' : '', 
        'label' : 'Set P3?', 
        'default_value' : True, 
    },    
    'choice3': {
        'type' : 'Select', 
        'description' : '', 
        'label' : 'Point 3', 
        'default_value' : 'Gamma', 
        'options' : {'Gamma':'Gamma','L    ':'L    ','X    ':'X    ','K    ':'K    ','W    ':'W    ','U    ':'U    '}, 
    },
    'check4': {
        'type' : 'Boolean', 
        'description' : '', 
        'label' : 'Set P4?', 
        'default_value' : False, 
    },
    'choice4': {
        'type' : 'Select', 
        'description' : '', 
        'label' : 'Point 4', 
        'default_value' : 'Gamma', 
        'options' : {'Gamma':'Gamma','L    ':'L    ','X    ':'X    ','K    ':'K    ','W    ':'W    ','U    ':'U    '}, 
    },
    'check5': {
        'type' : 'Boolean', 
        'description' : '', 
        'label' : 'Set P5?', 
        'default_value' : False, 
    },
    'choice5': {
        'type' : 'Select', 
        'description' : '', 
        'label' : 'Point 5', 
        'default_value' : 'Gamma', 
        'options' : {'Gamma':'Gamma','L    ':'L    ','X    ':'X    ','K    ':'K    ','W    ':'W    ','U    ':'U    '}, 
    },
    'check6': {
        'type' : 'Boolean', 
        'description' : '', 
        'label' : 'Set P6?', 
        'default_value' : False, 
    },
    'choice6': {
        'type' : 'Select', 
        'description' : '', 
        'label' : 'Point 6', 
        'default_value' : 'Gamma', 
        'options' : {'Gamma':'Gamma','L    ':'L    ','X    ':'X    ','K    ':'K    ','W    ':'W    ','U    ':'U    '}, 
    },
    'basis': {
        'type' : 'Select', 
        'id' : 'basis', 
        'description' : '', 
        'label' : 'K-Space Basis', 
        'default_value' : 'reciprocal', 
        'options' : {'reciprocal':'Reciprocal', 'cartesian':'Cartesian'}, 
    },
    'point_1': {
        'type' : 'String', 
        'description' : '', 
        'label' : 'Point 1',
        'default_value' : '', 
    },
    'point_2': {
        'type' : 'String', 
        'description' : '', 
        'label' : 'Point 2',
        'default_value' : '', 
    },
    'point_3': {
        'type' : 'String', 
        'description' : '', 
        'label' : 'Point 3',
        'default_value' : '', 
    },
    'point_4': {
        'type' : 'String', 
        'description' : '', 
        'label' : 'Point 4',
        'default_value' : '', 
    },
    'point_5': {
        'type' : 'String', 
        'description' : '', 
        'label' : 'Point 5',
        'default_value' : '', 
    },
    'point_6': {
        'type' : 'String', 
        'description' : '', 
        'label' : 'Point 6',
        'default_value' : '', 
    },    
    'nk_per_dir': {
        'type' : 'Integer', 
        'id' : 'nk_per_dir', 
        'description' : '', 
        'label' : 'k-points in every direction', 
        'default_value' : '11', 
        'min' : 2, 
        'max' : 41, 
        "units" : "",         
    },
    'kxmin': {
        'type' : 'Number', 
        'id' : 'kxmin', 
        'description' : '', 
        'label' : 'kmin', 
        'default_value' : '-0.5', 
        'min' : -1.0, 
        'max' : 1.0, 
        "units" : "",         
    },
    'kxmax': {
        'type' : 'Number', 
        'id' : 'kxmax', 
        'description' : '', 
        'label' : 'kmax', 
        'default_value' : '0.5', 
        'min' : -1.0, 
        'max' : 1.0, 
        "units" : "",         
    },
    'kymin': {
        'type' : 'Number', 
        'id' : 'kymin', 
        'description' : '', 
        'label' : 'kmin', 
        'default_value' : '-0.5', 
        'min' : -1.0, 
        'max' : 1.0, 
        "units" : "",         
    },
    'kymax': {
        'type' : 'Number', 
        'id' : 'kymax', 
        'description' : '', 
        'label' : 'kmax', 
        'default_value' : '0.5', 
        'min' : -1.0, 
        'max' : 1.0, 
        "units" : "",         
    },
    'kzmin': {
        'type' : 'Number', 
        'id' : 'kzmin', 
        'description' : '', 
        'label' : 'kmin', 
        'default_value' : '-0.5', 
        'min' : -1.0, 
        'max' : 1.0, 
        "units" : "",         
    },
    'kzmax': {
        'type' : 'Number', 
        'id' : 'kzmax', 
        'description' : '', 
        'label' : 'kmax', 
        'default_value' : '0.5', 
        'min' : -1.0, 
        'max' : 1.0, 
        "units" : "",         
    },
    #'3d': {
    #    'type' : 'Select', 
    #    'id' : '3d', 
    #    'description' : '', 
    #    'label' : '3D visualization', 
    #    'default_value' : 'cb', 
    #    'options' : { 'cb':'Conduction band', 'hh':'Heavy hole valence band', 'lh':'Light hole valence band', 'so':'Split-off valence band' }, 
    #},
    'kdir': {
        'type' : 'Select', 
        'id' : 'kdir', 
        'description' : 'Compute energy bands along this direction in k-space.', 
        'label' : 'Choose Path:', 'default_value' : 'X1_utb', 
        'options' : {
            'X1_utb':'(0,0) -> (0.5,0)',
            'X2_utb':'(0,0) -> (0,0.5)',
            'L1_utb':'(0,0) -> (0.5,0.5)',
            'L2_utb':'(0,0) -> (0.5,-0.5)',
            'full':'Full domain'
        }, 
    },
    'Nk': {
        'type' : 'Integer', 
        'id' : 'Nk', 'description' : 'Specify the k-point discretization between every corner point of the path in k-space.', 
        'label' : 'Number of k-points', 
        'default_value' : '30', 
        'min' : 2, 
        'max' : 151, 
        'units' : '',
    },
    'nbands': {
        'type' : 'Integer', 
        'id' : 'nbands', 
        'description' : 'Limit the number of bands computed to this number on the conduction or valence side of the gap.', 
        'label' : 'Number of bands', 
        'default_value' : '10', 
        'min' : 1, 
        'max' : 30, 
        'units' : "",
    },
    'user_override': {
        'type' : 'Boolean', 
        'id' : 'user_override', 
        'description' : 'Manually select a simulation venue rather than automatic selection by the tool.', 
        'label' : 'Manual simulation venue selection', 
        'default_value' : False,
    },
    'venue': {
        'type' : 'Select', 
        'id' : 'venue', 
        'description' : ' Select the computer cluster to run your simulations', 
        'label' : 'Simulation Venue', 
        'default_value' : 'local', 
        'options' : {'local':'nanoHUB.org (local)', 'nanohub':'nanoHUB.org (cluster)', 'RCAC':'RCAC'},
    }
}

## Defining Input Layout

In [10]:

        
LAYOUT = {
    'input': {
        'type': 'group',
        'id': '',
        'label': 'Bandstructure Details',
        'layout': 'horizontal',
        'children': [
            { 'type': '', 'id': 'geom', 'label': ''},
            { 'type': '', 'id': 'Type_of_calculation', 'label': ''},
            { 'type': '', 'id': 'material', 'label': ''},
            { 'type': '', 'id': 'alloy_model', 'label': '', 'enable': [
              {'operand': 'material', 'operator': 'in', 'value': "AlGaAs,SiGe"}
            ]},
            { 'type': '', 'id': 'molar_fraction', 'label': '', 'enable': [
              {'operand': 'material', 'operator': 'in', 'value': "AlGaAs,SiGe"}
            ]},
            { 'type': '', 'id': 'tbinding', 'label': '', 'enable': [
              {'operand': 'material', 'geom': 'in', 'value': "utb,wire"},
              {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Electron"},
            ]},
            { 'type': 'tab', 'label': '', 'enable': None, 'layout': 'horizontal', 'children': [
                { 'type': 'group', 'label': 'Physics', 'enable': None, 'layout': 'horizontal', 'children': [
                    { 'type': '', 'id': 'pmodel1', 'label': '', 'enable': [
                      {'operand': 'material', 'geom': 'in', 'value': "utb,wire,bulk"},
                      {'operand': 'material', 'operator': 'in', 'value': "Si,GaAs"},
                      {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Phonon"},
                    ]},
                    { 'type': '', 'id': 'pmodel2', 'label': '', 'enable': [
                      {'operand': 'material', 'geom': 'in', 'value': "utb,wire,bulk"},
                      {'operand': 'material', 'operator': 'in', 'value': "AlGaAs,SiGe,InAs"},
                      {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Phonon"},
                    ]},
                    { 'type': '', 'id': 'spin', 'label': '', 'enable': [
                      {'operand': 'material', 'geom': 'in', 'value': "utb,wire,bulk"},
                      {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Electron"},
                    ]},
                    { 'type': '', 'id': 'model', 'label': '', 'enable': [
                      {'operand': 'material', 'geom': 'in', 'value': "utb,wire,bulk"},
                      {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Electron"},
                    ]},
                    { 'type': '', 'id': 'sweep', 'label': '', 'enable': [
                      {'operand': 'material', 'geom': 'in', 'value': "utb,wire,bulk"},
                      {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Electron"},
                      {'operand': 'model', 'operator': 'in', 'value': "hydrostatic,uniaxial,biaxial"},
                    ]},
                    { 'type': '', 'id': 'epsilon', 'label': '', 'enable': [
                      {'operand': 'material', 'geom': 'in', 'value': "utb,wire,bulk"},
                      {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Electron"},
                      {'operand': 'model', 'operator': 'in', 'value': "hydrostatic,uniaxial,biaxial"},
                      {'operand': 'sweep', 'operator': '==', 'value': "false"},
                    ]},
                    { 'type': '', 'id': 'eps1', 'label': '', 'enable': [
                      {'operand': 'material', 'geom': 'in', 'value': "utb,wire,bulk"},
                      {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Electron"},
                      {'operand': 'model', 'operator': 'in', 'value': "hydrostatic,uniaxial,biaxial"},
                      {'operand': 'sweep', 'operator': '==', 'value': "true"},
                    ]},
                    { 'type': '', 'id': 'eps2', 'label': '', 'enable': [
                      {'operand': 'material', 'geom': 'in', 'value': "utb,wire,bulk"},
                      {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Electron"},
                      {'operand': 'model', 'operator': 'in', 'value': "hydrostatic,uniaxial,biaxial"},
                      {'operand': 'sweep', 'operator': '==', 'value': "true"},
                    ]},
                    { 'type': '', 'id': 'num_eps', 'label': '', 'enable': [
                      {'operand': 'material', 'geom': 'in', 'value': "utb,wire,bulk"},
                      {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Electron"},
                      {'operand': 'model', 'operator': 'in', 'value': "hydrostatic,uniaxial,biaxial"},
                      {'operand': 'sweep', 'operator': '==', 'value': "true"},
                    ]},
                    { 'type': '', 'id': 'x', 'label': '', 'enable': [
                      {'operand': 'material', 'geom': 'in', 'value': "utb,wire,bulk"},
                      {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Electron"},
                      {'operand': 'model', 'operator': 'in', 'value': "uniaxial,biaxial"},
                    ]},
                    { 'type': '', 'id': 'y', 'label': '', 'enable': [
                      {'operand': 'material', 'geom': 'in', 'value': "utb,wire,bulk"},
                      {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Electron"},
                      {'operand': 'model', 'operator': 'in', 'value': "uniaxial,biaxial"},
                    ]},
                    { 'type': '', 'id': 'z', 'label': '', 'enable': [
                      {'operand': 'material', 'geom': 'in', 'value': "utb,wire,bulk"},
                      {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Electron"},
                      {'operand': 'model', 'operator': 'in', 'value': "uniaxial,biaxial"},
                    ]},
                    { 'type': '', 'id': 'exx', 'label': '', 'enable': [
                      {'operand': 'material', 'geom': 'in', 'value': "utb,wire,bulk"},
                      {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Electron"},
                      {'operand': 'model', 'operator': '==', 'value': "freeform"},
                    ]},
                    { 'type': '', 'id': 'eyy', 'label': '', 'enable': [
                      {'operand': 'material', 'geom': 'in', 'value': "utb,wire,bulk"},
                      {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Electron"},
                      {'operand': 'model', 'operator': '==', 'value': "freeform"},
                    ]},
                    { 'type': '', 'id': 'ezz', 'label': '', 'enable': [
                      {'operand': 'material', 'geom': 'in', 'value': "utb,wire,bulk"},
                      {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Electron"},
                      {'operand': 'model', 'operator': '==', 'value': "freeform"},
                    ]},
                    { 'type': '', 'id': 'exy', 'label': '', 'enable': [
                      {'operand': 'material', 'geom': 'in', 'value': "utb,wire,bulk"},
                      {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Electron"},
                      {'operand': 'model', 'operator': '==', 'value': "freeform"},
                    ]},
                    { 'type': '', 'id': 'exz', 'label': '', 'enable': [
                      {'operand': 'material', 'geom': 'in', 'value': "utb,wire,bulk"},
                      {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Electron"},
                      {'operand': 'model', 'operator': '==', 'value': "freeform"},
                    ]},
                    { 'type': '', 'id': 'eyz', 'label': '', 'enable': [
                      {'operand': 'material', 'geom': 'in', 'value': "utb,wire,bulk"},
                      {'operand': 'Type_of_calculation', 'operator': '==', 'value': "Electron"},
                      {'operand': 'model', 'operator': '==', 'value': "freeform"},
                    ]},
                    #{ 'type': '', 'id': 'which_bands', 'label': '', 'enable': [
                    #  {'operand': 'job ', 'operator': 'in', 'value': "UTBEK,WBS"},    #Never visible                     
                    #]},
                    #{ 'type': '', 'id': 'shift', 'label': '', 'enable': [
                    #  {'operand': 'geom', 'operator': 'in', 'value': "wire,utb"},
                    #  {'operand': 'which_bands', 'operator': '==', 'value': "manual"},                        
                    #]},
                    { 'type': '', 'id': 'dangling', 'label': '', 'enable': [
                      {'operand': 'geom', 'operator': 'in', 'value': "wire,utb"},
                    ]},
                    { 'type': '', 'id': 'eps', 'label': '', 'enable': [
                      {'operand': 'geom', 'operator': 'in', 'value': "wire,utb"},
                    ]},        
                    { 'type': '', 'id': 'Tel', 'label': '', 'enable': [
                      {'operand': 'geom', 'operator': 'in', 'value': "wire,utb"},
                    ]},
                    { 'type': '', 'id': 'gate_sweep', 'label': '', 'enable': [
                      {'operand': 'geom', 'operator': 'in', 'value': "wire,utb"},
                    ]},
                    { 'type': 'group', 'label': 'Front gate bias(V)', 'enable': [
                      {'operand': 'gate_sweep', 'operator': '==', 'value': True},
                    ], 'layout': 'horizontal', 'children': [
                        { 'type': '', 'id': 'VG', 'label': ''}, 
                        { 'type': '', 'id': 'vgmin', 'label': ''}, 
                        { 'type': '', 'id': 'vgmax', 'label': ''}, 
                        { 'type': '', 'id': 'vgdiv', 'label': ''}, 
                    ]},
                ]},
                { 'type': 'group', 'label': 'TODO', 'enable': None, 'layout': 'horizontal', 'children': [
                    { 'type': '', 'id': 'VD', 'label': ''}, 
                    { 'type': '', 'id': 'workfunction', 'label': ''}, 
                    { 'type': '', 'id': 'el_aff', 'label': ''}, 
                    { 'type': '', 'id': 'config', 'label': ''}, 
                    { 'type': '', 'id': 'Vback', 'label': ''}, 
                    { 'type': '', 'id': 'EF_method', 'label': ''}, 
                    { 'type': '', 'id': 'EF', 'label': ''}, 
                    { 'type': '', 'id': 'choice_Symmetry_point', 'label': ''}, 
                    { 'type': '', 'id': 'path', 'label': ''}, 
                    { 'type': '', 'id': 'choice1', 'label': ''}, 
                    { 'type': '', 'id': 'L', 'label': ''}, 
                    { 'type': '', 'id': 'X', 'label': ''}, 
                    { 'type': '', 'id': 'K', 'label': ''}, 
                    { 'type': '', 'id': 'W', 'label': ''}, 
                    { 'type': '', 'id': 'U', 'label': ''}, 
                    { 'type': '', 'id': 'choice2', 'label': ''},
                    { 'type': '', 'id': 'check3', 'label': ''},
                    { 'type': '', 'id': 'choice3', 'label': ''},
                    { 'type': '', 'id': 'check4', 'label': ''},
                    { 'type': '', 'id': 'choice4', 'label': ''},
                    { 'type': '', 'id': 'check5', 'label': ''},
                    { 'type': '', 'id': 'choice5', 'label': ''},
                    { 'type': '', 'id': 'check6', 'label': ''},
                    { 'type': '', 'id': 'choice6', 'label': ''},
                    { 'type': '', 'id': 'point_1', 'label': ''},
                    { 'type': '', 'id': 'point_2', 'label': ''},
                    { 'type': '', 'id': 'point_3', 'label': ''},
                    { 'type': '', 'id': 'point_4', 'label': ''},
                    { 'type': '', 'id': 'point_5', 'label': ''},
                    { 'type': '', 'id': 'point_6', 'label': ''},
                    { 'type': '', 'id': 'kxmin', 'label': ''},
                    { 'type': '', 'id': 'kxmax', 'label': ''},
                    { 'type': '', 'id': 'kymin', 'label': ''},
                    { 'type': '', 'id': 'kymax', 'label': ''},
                    { 'type': '', 'id': 'kzmin', 'label': ''},
                    { 'type': '', 'id': 'kzmax', 'label': ''},
                    #{ 'type': '', 'id': '3d', 'label': '', 'enable': [
                    #  {'operand': 'geom', 'operator': '==', 'value': "bulk"}
                    #]},
                    { 'type': '', 'id': 'kdir', 'label': ''},
                    { 'type': '', 'id': 'Nk', 'label': ''},
                    { 'type': '', 'id': 'nbands', 'label': ''},
                    { 'type': '', 'id': 'user_override', 'label': ''},
                    { 'type': '', 'id': 'venue', 'label': ''},      
                ]},
            ]},
        ]
    }
}
#LAYOUT

## Creating the Parameters/Inputs panel

In [11]:
url_sim = "https://nanohub.org/api/tools"
NsopticsSettings = AppBuilder.Settings(
    Project,
    Component, 
    SETTINGS,
    url=url_sim,
    toolname = TOOLNAME,
    layout = LAYOUT['input']
)
NsopticsSettings.content.events["onError"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False},
    { "type": "stateChange", "modifies": STATE_ERROR_OPEN,"newState": True},
    { "type": "stateChange", "modifies": STATE_ERROR_STATUS,"newState": '$e'},
]
NsopticsSettings.content.events["click"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": True}
]
NsopticsSettings.content.events["submit"] = [
    { "type": "stateChange", "modifies": "parameters","newState": '$e.target.value'}
]


## Defining output parameters

In [12]:
RapptureBuilder.loadXY(Project, Component)
RapptureBuilder.loadSequence(Project, Component)
RESULTS = {
    "ESC" : {
        'title' : 'Extinction Cross Section',
        'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['ESC']"] },
    },
    "SCA" : {
        'title' : 'Scattering Cross Section',
        'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['SCA']"] },
    },
    "ABS" : {
        'title' : 'Absortion Cross section',
        'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['ABS']"] },
    },
    "REP" : {
        'title' : 'Real(Dielectric Constant)',
        'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['REP']"] },
    },
    "IEP" : {
        'title' : 'Imaginary(Dielectric Constant)',
        'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['IEP']"] },
    },
}

NsopticsResults = AppBuilder.Results( 
    Component,
    results = RESULTS,
    onClick = [{ "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": True }],
    onLoad = [{ "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False }],
)
NsopticsSettings.content.events["onStatusChange"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_STATUS,"newState": "$e.target.value"}
]
NsopticsSettings.content.events["onSuccess"]=[
  { "type": "stateChange", "modifies": "open_plot","newState":{'ESC':'primary'} },
  { "type": "propCall2", "calls": "loadXY","args": ["self", "['ESC']"]},
  { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False },  
  { "type": "stateChange", "modifies": "open_results","newState": True},
  { "type": "stateChange", "modifies": "open_params","newState": False},
]

## Creating Colorschema and adding nanoHUB logo

In [13]:
Theme = MaterialBuilder.DefaultTheme(
    primary_color = '#699FBB',
    secondary_color = '#f1f1f1',
    primary_bg = '#FFFFFF',
    secondary_bg = '#dbeaf0',
    default_button = 'rgba(255, 255, 255, 0.87)',
    primary_button = 'rgba(255, 255, 255, 0.87)',
    secondary_button = 'rgba(0, 0, 0, 0.87)',
    default_button_bg = 'rgb(63, 162, 192)',
    primary_button_bg = 'rgba(0, 0, 0, 0.65)',
    secondary_button_bg = 'rgba(0, 0, 0, 0.12)',    
    drawer_position = "relative"
)
ThemeProvider = MaterialBuilder.ThemeProvider( Component, Theme )
Theme['components']['MuiDrawer']['styleOverrides']['paper']['margin-top'] = '0px'
Theme['components']['MuiDrawer']['styleOverrides']['paper']['height'] = '100%'
AppBar = MaterialBuilder.AppBar(
    state="DrawerIsVisible", 
    title="Nanosphere Optics Lab"
)

logo = t.TeleportElement(t.TeleportContent(elementType="img"))
logo.content.attrs["width"] = "120"
logo.content.attrs["src"] = "https://nanohub.org/app/site/media/images/PressKit/nanoHUB_logo_color.jpg"
AppBar.content.children[0].addContent(logo)

## Creating Collapsable panels, for input and output parameters 

In [14]:
ExpansionPanel = MaterialBuilder.ExpansionPanel(
    title="Results", 
    content=[NsopticsResults],
    expanded={
      "type": "dynamic",
      "content": {
        "referenceType": "state",
        "id": "open_results"
      }    
    },     
)
ExpansionPanel.content.events['change'] = [{ "type": "stateChange", "modifies": "open_results","newState": "$toggle"}]
Drawer.addContent(ExpansionPanel)

ParametersPanel = MaterialBuilder.ExpansionPanel(
    title="Parameters", 
    expanded={
      "type": "dynamic",
      "content": {
        "referenceType": "state",
        "id": "open_params"
      }    
    }, 
    content=[NsopticsSettings]
)
ParametersPanel.content.events['change'] = [{ "type": "stateChange", "modifies": "open_params","newState": "$toggle"}]
Drawer.addContent(ParametersPanel)


## Creating Reset button

In [15]:
Gridt = t.TeleportElement(MaterialContent(elementType="Grid"))
Gridt.content.attrs["color"] = "secondary"
Gridt.content.attrs["container"] = True
Gridt.content.attrs["direction"] = "column"
resetSettings = {}
for k,v in SETTINGS.items():
    resetSettings[k] = v["default_value"]

Buttontt= MaterialBuilder.Button(
      title = "Reset Setting", 
      variant = "text", 
      style = {'backgroundColor':'#999999', 'borderRadius' : '0px', 'minHeight':'40px', 'color':'rgba(255, 255, 255, 0.87)'}, 
      onClickButton=[{ "type": "stateChange", "modifies": "parameters","newState": resetSettings}]
)
Gridt.addContent(Buttontt)
Drawer.addContent(Gridt)

## Creating ploting component

In [16]:
BasePlot = PlotlyBuilder.BasePlot( Project, Component)

## Assembling the app

In [17]:
Gridh = t.TeleportElement(MaterialContent(elementType="Grid"))
Gridh.content.attrs["container"] = True
Gridh.content.attrs["direction"] = "row" 
Gridh.addContent(Drawer)
Gridh.addContent(BasePlot)  

Gridv = t.TeleportElement(MaterialContent(elementType="Grid"))
Gridv.content.attrs["container"] = True
Gridv.content.attrs["direction"] = "column"
Gridv.addContent(AppBar)
Gridv.addContent(Gridh)


ThemeProvider.addContent(Gridv)
ThemeProvider.addContent(Loader)
ThemeProvider.addContent(ErrorMessage)
ThemeProvider.addContent(Login)

Component.addNode(ThemeProvider)
Project.buildReact(
    TOOLNAME.upper().replace("_","")+".HTML", 
    copy_libraries=False, 
    run_uidl="redirect", 
    jupyter_notebook_url=jupyter_notebook_url 
);

building HTML 
done!
https://proxy.nanohub.org/weber/2163531/4zR6ixKMr1itlvZR/23/uidl/BANDSTRLAB.HTML/redirect/nanohub-uidl/examples/tools/BANDSTRLAB.HTML


## Displaying the app in jupyter

In [18]:
from nanohubuidl.ipywidgets import buildWidget
BasePlot.content.style = {
  "position": "absolute",
  "height": "700px",
  "width": "100%",
}
Drawer.content.style = {
  "height": "700px",
}
MyComponentWidget = buildWidget(Project, jupyter_axios=True, debugger=True, verbose=False);

In [19]:
a = MyComponentWidget()

HTML(value="<link rel='stylesheet' href='https://fonts.googleapis.com/icon?family=Material+Icons'/><style>.p-W…

<IPython.core.display.Javascript object>

In [20]:
a

BandstructuresLabWidget(config={'displayModeBar': True, 'responsive': True}, data=[{'x': [], 'y': []}], parame…